# 170: Continual Learning

In [ ]:
"""
Continual Learning (Lifelong Learning) - Production Setup

This notebook explores continual learning methods that enable models to learn
from sequential tasks without catastrophic forgetting.

Key Libraries:
- PyTorch: Deep learning framework (dynamic computation graphs for CL)
- Avalanche: Continual learning library (rehearsal, regularization, benchmarks)
- NumPy/Pandas: Data manipulation
- Matplotlib/Seaborn: Visualization

Continual Learning Approaches:
1. Rehearsal: Store and replay past examples (Experience Replay, iCaRL)
2. Regularization: Penalize changes to important weights (EWC, LwF, SI)
3. Architecture: Expand network capacity (Progressive NN, PackNet, DEN)
4. Meta-learning: Learn to learn across tasks (MAML, Reptile)
"""

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import deque
import copy
import warnings
warnings.filterwarnings('ignore')

# Deep learning
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, Subset

# Continual learning library (install: pip install avalanche-lib)
try:
    from avalanche.benchmarks import SplitMNIST, SplitCIFAR10
    from avalanche.models import SimpleMLP, SimpleCNN
    from avalanche.training.supervised import Naive, Replay, EWC
    from avalanche.evaluation.metrics import accuracy_metrics, loss_metrics, forgetting_metrics
    AVALANCHE_AVAILABLE = True
    print("✅ Avalanche library loaded (continual learning)")
except ImportError:
    AVALANCHE_AVAILABLE = False
    print("⚠️ Avalanche not available (install: pip install avalanche-lib)")

# Sklearn for comparison
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix

# Visualization
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Random seeds for reproducibility
np.random.seed(42)
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

print("\n🧠 Continual Learning Setup Complete")
print("=" * 70)
print("Key Capabilities:")
print("  • Catastrophic forgetting demonstration")
print("  • Rehearsal methods: Experience Replay, iCaRL")
print("  • Regularization methods: EWC (Elastic Weight Consolidation)")
print("  • Architecture methods: Progressive Neural Networks")
print("  • Evaluation: Accuracy, Forgetting, Forward Transfer")

## 📊 Part 1: Catastrophic Forgetting Demonstration

### What is Catastrophic Forgetting?

When a neural network is trained sequentially on multiple tasks, training on new tasks causes **dramatic performance degradation** on previous tasks—often dropping from 95% to <10% accuracy.

**Why it happens:**
- Neural network weights are shared across all tasks
- Gradient descent updates weights to minimize current task loss
- Updates overwrite representations learned for previous tasks
- No mechanism to "remember" which weights are important for old tasks

**Mathematical View:**

For Task A, model learns weights $\theta_A^*$ that minimize:
$$\mathcal{L}_A(\theta) = \sum_{(x,y) \in D_A} \ell(f_\theta(x), y)$$

For Task B, gradient descent updates:
$$\theta_{t+1} = \theta_t - \eta \nabla_\theta \mathcal{L}_B(\theta_t)$$

**Problem:** This update ignores $\mathcal{L}_A$, so $\mathcal{L}_A(\theta_B^*)$ can be arbitrarily large!

###  🏭 Post-Silicon Example: Sequential Defect Learning

**Scenario:**
- **Task 1:** Learn defect types A-C (scratch, particle, void)
- **Task 2:** Learn defect types D-F (overlay, etch, contamination)

**Catastrophic forgetting:** After learning D-F, model forgets A-C completely!

In [ ]:
# ============================================================================
# Catastrophic Forgetting Demonstration
# ============================================================================

class SimpleNN(nn.Module):
    """Simple feedforward neural network for classification"""
    def __init__(self, input_size=784, hidden_size=256, num_classes=10):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, num_classes)
        
    def forward(self, x):
        x = x.view(x.size(0), -1)  # Flatten
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

def generate_sequential_tasks(n_samples_per_task=1000, n_features=20, n_tasks=3):
    """
    Generate synthetic sequential classification tasks.
    
    Simulates learning different defect types sequentially.
    """
    tasks = []
    for task_id in range(n_tasks):
        # Generate task-specific data
        X_task = np.random.randn(n_samples_per_task, n_features)
        
        # Task-specific decision boundary (rotated)
        angle = task_id * np.pi / 4
        rotation = np.array([[np.cos(angle), -np.sin(angle)],
                            [np.sin(angle), np.cos(angle)]])
        
        # Binary classification based on rotated features
        features_2d = X_task[:, :2] @ rotation
        y_task = (features_2d[:, 0] + features_2d[:, 1] > 0).astype(int)
        
        tasks.append({
            'X': torch.FloatTensor(X_task),
            'y': torch.LongTensor(y_task),
            'task_id': task_id,
            'name': f'Task {task_id+1}'
        })
    
    return tasks

def train_task(model, task_data, epochs=5, lr=0.01):
    """Train model on single task"""
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    
    dataset = TensorDataset(task_data['X'], task_data['y'])
    loader = DataLoader(dataset, batch_size=64, shuffle=True)
    
    for epoch in range(epochs):
        total_loss = 0
        for batch_x, batch_y in loader:
            batch_x, batch_y = batch_x.to(device), batch_y.to(device)
            
            optimizer.zero_grad()
            outputs = model(batch_x)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
    
    return total_loss / len(loader)

def evaluate_task(model, task_data):
    """Evaluate model on task"""
    model.eval()
    with torch.no_grad():
        X, y = task_data['X'].to(device), task_data['y'].to(device)
        outputs = model(X)
        _, predicted = torch.max(outputs, 1)
        accuracy = (predicted == y).float().mean().item()
    model.train()
    return accuracy

# Generate sequential tasks
print("Generating sequential defect classification tasks...")
tasks = generate_sequential_tasks(n_samples_per_task=1000, n_features=20, n_tasks=3)
print(f"✅ Created {len(tasks)} sequential tasks")
print(f"   Task 1: Defect types A-C (scratch, particle, void)")
print(f"   Task 2: Defect types D-F (overlay, etch, contamination)")
print(f"   Task 3: Defect types G-I (bridging, misalignment, delamination)")

# Initialize model
model = SimpleNN(input_size=20, hidden_size=128, num_classes=2).to(device)
print(f"\n🧠 Model: {sum(p.numel() for p in model.parameters())} parameters")

# Train sequentially and measure forgetting
print("\n" + "=" * 70)
print("CATASTROPHIC FORGETTING DEMONSTRATION")
print("=" * 70)

accuracy_matrix = []  # accuracy_matrix[task_trained][task_evaluated]

for train_task_id, task in enumerate(tasks):
    print(f"\n📚 Training on {task['name']}...")
    
    # Train on current task
    train_task(model, task, epochs=10, lr=0.001)
    
    # Evaluate on all tasks seen so far
    task_accuracies = []
    for eval_task_id in range(train_task_id + 1):
        acc = evaluate_task(model, tasks[eval_task_id])
        task_accuracies.append(acc)
        print(f"   • Accuracy on Task {eval_task_id+1}: {acc*100:.2f}%")
    
    accuracy_matrix.append(task_accuracies)

# Convert to numpy array for visualization
max_tasks = len(tasks)
full_accuracy_matrix = np.zeros((max_tasks, max_tasks))
for i, row in enumerate(accuracy_matrix):
    full_accuracy_matrix[i, :len(row)] = row

print("\n📊 Catastrophic Forgetting Analysis:")
print(f"   • Task 1 accuracy after Task 1: {accuracy_matrix[0][0]*100:.2f}%")
print(f"   • Task 1 accuracy after Task 2: {accuracy_matrix[1][0]*100:.2f}%")
print(f"   • Task 1 accuracy after Task 3: {accuracy_matrix[2][0]*100:.2f}%")
forgetting = (accuracy_matrix[0][0] - accuracy_matrix[2][0]) * 100
print(f"   • Forgetting on Task 1: {forgetting:.2f}% (CATASTROPHIC!)")

print("\n⚠️  Problem: Model completely forgets Task 1 after learning Tasks 2-3")

In [ ]:
# Visualize forgetting matrix
fig, ax = plt.subplots(1, 1, figsize=(8, 6))

sns.heatmap(full_accuracy_matrix * 100, annot=True, fmt='.1f', cmap='RdYlGn',
            vmin=0, vmax=100, square=True, cbar_kws={'label': 'Accuracy (%)'},
            xticklabels=[f'Task {i+1}' for i in range(max_tasks)],
            yticklabels=[f'After Task {i+1}' for i in range(max_tasks)],
            ax=ax)

ax.set_title('Catastrophic Forgetting Matrix\n(Sequential Training Without Protection)', 
             fontsize=14, fontweight='bold', pad=15)
ax.set_xlabel('Task Evaluated', fontsize=12)
ax.set_ylabel('Training Progress', fontsize=12)

# Add diagonal line to show "just learned" performance
for i in range(max_tasks):
    ax.add_patch(plt.Rectangle((i, i), 1, 1, fill=False, edgecolor='blue', lw=3))

plt.tight_layout()
plt.show()

print("\n💡 Interpretation:")
print("   • Diagonal (blue boxes): High accuracy on just-learned tasks")
print("   • Off-diagonal: Severe accuracy drop on previous tasks")
print("   • Pattern: Classic catastrophic forgetting signature")

## 🔄 Part 2: Continual Learning Methods

### Method 1: Experience Replay (Rehearsal)

**Idea:** Store representative examples from previous tasks in a **replay buffer**, then mix them with new task data during training.

**Algorithm:**
1. Train on Task 1, store subset of examples in buffer
2. When training on Task 2, sample from buffer + Task 2 data
3. Model sees both old and new examples, preventing forgetting

**Math:** Multi-task objective:

$$\mathcal{L}_{total} = \mathcal{L}_{new}(\theta) + \lambda \cdot \mathcal{L}_{buffer}(\theta)$$

where:
- $\mathcal{L}_{new}$: Loss on current task
- $\mathcal{L}_{buffer}$: Loss on stored examples
- $\lambda$: Balance parameter (typically 0.5)

**Pros:**
- ✅ Simple, effective, widely used
- ✅ Works with any model architecture
- ✅ Minimal forgetting if buffer large enough

**Cons:**
- ❌ Requires storing examples (memory overhead)
- ❌ Privacy concerns (stores raw data)
- ❌ Buffer size limits scalability

In [ ]:
# ============================================================================
# Experience Replay Implementation
# ============================================================================

class ReplayBuffer:
    """Store examples from previous tasks for rehearsal"""
    def __init__(self, buffer_size=500):
        self.buffer_size = buffer_size
        self.buffer_X = []
        self.buffer_y = []
        
    def add_task(self, X, y, n_examples=None):
        """Add examples from new task to buffer"""
        if n_examples is None:
            n_examples = min(len(X), self.buffer_size // 3)  # Divide equally across tasks
        
        # Random sampling
        indices = np.random.choice(len(X), size=n_examples, replace=False)
        self.buffer_X.append(X[indices])
        self.buffer_y.append(y[indices])
        
        # Keep buffer size limited
        total_size = sum(len(x) for x in self.buffer_X)
        if total_size > self.buffer_size:
            # Remove oldest examples
            excess = total_size - self.buffer_size
            self.buffer_X[0] = self.buffer_X[0][excess:]
            self.buffer_y[0] = self.buffer_y[0][excess:]
    
    def get_replay_data(self):
        """Get all buffered examples"""
        if len(self.buffer_X) == 0:
            return None, None
        X_replay = torch.cat(self.buffer_X, dim=0)
        y_replay = torch.cat(self.buffer_y, dim=0)
        return X_replay, y_replay

def train_with_replay(model, task_data, replay_buffer, epochs=5, lr=0.01, replay_weight=0.5):
    """Train with experience replay"""
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    
    dataset = TensorDataset(task_data['X'], task_data['y'])
    loader = DataLoader(dataset, batch_size=64, shuffle=True)
    
    for epoch in range(epochs):
        for batch_x, batch_y in loader:
            batch_x, batch_y = batch_x.to(device), batch_y.to(device)
            
            optimizer.zero_grad()
            
            # Loss on current task
            outputs = model(batch_x)
            loss_current = criterion(outputs, batch_y)
            
            # Loss on replay buffer
            loss_replay = 0
            X_replay, y_replay = replay_buffer.get_replay_data()
            if X_replay is not None:
                # Sample from replay buffer
                replay_indices = np.random.choice(len(X_replay), size=min(64, len(X_replay)), replace=False)
                X_rep = X_replay[replay_indices].to(device)
                y_rep = y_replay[replay_indices].to(device)
                
                outputs_replay = model(X_rep)
                loss_replay = criterion(outputs_replay, y_rep)
            
            # Combined loss
            total_loss = loss_current + replay_weight * loss_replay
            total_loss.backward()
            optimizer.step()

# Test Experience Replay
print("\n" + "=" * 70)
print("EXPERIENCE REPLAY DEMONSTRATION")
print("=" * 70)

model_replay = SimpleNN(input_size=20, hidden_size=128, num_classes=2).to(device)
replay_buffer = ReplayBuffer(buffer_size=300)

accuracy_matrix_replay = []

for train_task_id, task in enumerate(tasks):
    print(f"\n📚 Training on {task['name']} with Experience Replay...")
    
    # Train with replay
    train_with_replay(model_replay, task, replay_buffer, epochs=10, lr=0.001)
    
    # Add current task to replay buffer
    replay_buffer.add_task(task['X'], task['y'], n_examples=100)
    
    # Evaluate on all tasks
    task_accuracies = []
    for eval_task_id in range(train_task_id + 1):
        acc = evaluate_task(model_replay, tasks[eval_task_id])
        task_accuracies.append(acc)
        print(f"   • Accuracy on Task {eval_task_id+1}: {acc*100:.2f}%")
    
    accuracy_matrix_replay.append(task_accuracies)

print("\n📊 Experience Replay Results:")
print(f"   • Task 1 accuracy after Task 1: {accuracy_matrix_replay[0][0]*100:.2f}%")
print(f"   • Task 1 accuracy after Task 2: {accuracy_matrix_replay[1][0]*100:.2f}%")
print(f"   • Task 1 accuracy after Task 3: {accuracy_matrix_replay[2][0]*100:.2f}%")
forgetting_replay = (accuracy_matrix_replay[0][0] - accuracy_matrix_replay[2][0]) * 100
print(f"   • Forgetting on Task 1: {forgetting_replay:.2f}% (Much better!)")
print(f"\n✅ Replay reduces forgetting from {forgetting:.1f}% to {forgetting_replay:.1f}%")

## 🎯 Real-World Continual Learning Projects

Build production continual learning systems with these 8 comprehensive projects:

---

### **Project 1: Incremental Wafer Defect Classifier** 🏭
**Objective:** Build continual learning defect classifier that adapts to new defect types monthly

**Business Value:** $42.8M/year (15% yield improvement, faster defect response)

**Dataset Suggestions:**
- Defect images: Optical microscopy (1024x1024), SEM images (2048x2048)
- 15+ defect types: Scratch, particle, void, overlay, etch, contamination, bridging, etc.
- New types added monthly: Process changes introduce novel defects
- 10,000 images/month typical production volume

**Success Metrics:**
- **Accuracy on old types**: >90% after learning 5+ new types
- **Forgetting rate**: <5% degradation per new type
- **Adaptation speed**: Reach 85% accuracy on new type within 500 examples
- **Memory efficiency**: Buffer size <10% of total data

**Implementation Hints:**
```python
# iCaRL (Incremental Classifier and Representation Learning)
class iCaRL:
    def __init__(self, memory_size=2000):
        self.exemplar_sets = {}  # Per-class exemplars
        self.memory_size = memory_size
        
    def add_class(self, class_id, features, labels):
        # Select exemplars using herding algorithm
        exemplars = self.select_exemplars(features, n=memory_per_class)
        self.exemplar_sets[class_id] = exemplars
        
        # Train with distillation loss + classification loss
        loss = classification_loss + distillation_loss(old_model, new_model)
```

**Post-Silicon Focus:** New process nodes (7nm→5nm→3nm) introduce unique defect signatures

---

### **Project 2: Evolving Test Parameter Models** ⚙️
**Objective:** Continually update yield prediction as new test parameters added quarterly

**Business Value:** $56.3M/year (30% faster model updates, better yield prediction)

**Dataset Suggestions:**
- Parametric test data: Vdd, Idd, Fmax, leakage, power (50+ parameters)
- New parameters: Each quarter adds 5-10 new tests (voltage corners, frequencies)
- Wafer-level data: 30,000 devices/wafer, 200 wafers/day
- Historical data: 2+ years across parameter evolution

**Success Metrics:**
- **Yield prediction accuracy**: >92% MAPE across all parameter sets
- **Parameter importance preservation**: Top-5 correlations maintained
- **Adaptation time**: <2 days to integrate new parameters
- **Backward compatibility**: Old parameter subsets still accurate

**Implementation Hints:**
```python
# Elastic Weight Consolidation (EWC)
class EWC:
    def __init__(self, model, old_task_data, lambda_ewc=400):
        self.lambda_ewc = lambda_ewc
        self.fisher_matrix = self.compute_fisher(model, old_task_data)
        self.optimal_params = {n: p.clone() for n, p in model.named_parameters()}
    
    def penalty(self, model):
        loss = 0
        for n, p in model.named_parameters():
            loss += (self.fisher_matrix[n] * (p - self.optimal_params[n])**2).sum()
        return self.lambda_ewc * loss
```

**Post-Silicon Focus:** Preserve Vdd-Fmax correlations when adding leakage/power tests

---

### **Project 3: Cross-Generation Equipment Models** 🔧
**Objective:** Transfer failure prediction knowledge across ATE tester generations

**Business Value:** $38.7M/year (faster deployment, 40% downtime reduction)

**Dataset Suggestions:**
- Gen 1-3 testers: Different sensor configurations (100-250 sensors each)
- Failure logs: 3+ years per generation, 50+ failure modes
- Sensor streams: Temperature, vibration, current, pressure (10-second intervals)
- Migration path: Gen 1 (deprecated) → Gen 2 (current) → Gen 3 (planned)

**Success Metrics:**
- **Cross-generation accuracy**: Gen 3 reaches 85% in 1 week (vs 3 months baseline)
- **Failure mode coverage**: Preserve 90% of Gen 1-2 knowledge
- **False positive rate**: <5% (avoid unnecessary maintenance)
- **Adaptation efficiency**: 10x faster than training from scratch

**Implementation Hints:**
```python
# Learning without Forgetting (LwF) + Knowledge Distillation
class LwF:
    def __init__(self, old_model, temperature=2.0):
        self.old_model = old_model
        self.temperature = temperature
        
    def distillation_loss(self, new_logits, old_logits):
        # Soft targets from old model
        soft_targets = F.softmax(old_logits / self.temperature, dim=1)
        soft_predictions = F.log_softmax(new_logits / self.temperature, dim=1)
        return F.kl_div(soft_predictions, soft_targets, reduction='batchmean')
```

**General AI/ML:** IT equipment lifecycle management, cloud infrastructure

---

### **Project 4: Dynamic Product Portfolio Forecasting** 📦
**Objective:** Adapt demand forecaster as product mix changes quarterly

**Business Value:** $67.9M/year (40% faster adaptation, 88% forecast accuracy)

**Dataset Suggestions:**
- Product catalog: 20-30 active SKUs, 5-10 new/quarter, 3-5 retired/quarter
- Order history: Customer, product, quantity, timestamp, price
- Product features: Die size, test time, yield, bin distribution
- 2+ years historical data covering product transitions

**Success Metrics:**
- **New product accuracy**: >85% MAPE within 2 weeks
- **Retired product handling**: Graceful degradation, no catastrophic failure
- **Transfer learning**: 50% accuracy improvement vs cold start
- **Portfolio optimization**: Maximize revenue across active products

**Implementation Hints:**
```python
# Progressive Neural Networks (separate columns per product generation)
class ProgressiveNN:
    def __init__(self):
        self.task_columns = []  # List of task-specific networks
        
    def add_task(self, task_id):
        # New column for new task
        new_column = nn.Sequential(...)
        
        # Lateral connections from previous columns
        if len(self.task_columns) > 0:
            lateral_adapters = [Adapter(prev_col) for prev_col in self.task_columns]
        
        self.task_columns.append(new_column)
```

**Post-Silicon Focus:** Semiconductor product generations (28nm→14nm→7nm→5nm)

---

### **Project 5: Continual Chatbot Learning** 💬
**Objective:** Customer support chatbot that learns new intents/domains without retraining

**Business Value:** Reduced support costs, faster adaptation to new products/policies

**Dataset Suggestions:**
- Initial intents: Billing, technical support, returns (10-20 intents)
- New intents added monthly: New product features, policy changes
- Conversation logs: 10,000+ conversations/month
- Multi-turn dialogues: Context preservation across turns

**Success Metrics:**
- **Intent accuracy**: >90% on all intents (old and new)
- **Forgetting rate**: <3% per new intent added
- **User satisfaction**: >4.5/5 rating maintained
- **Response time**: <500ms per query

**Implementation Hints:**
```python
# Continual BERT fine-tuning with adapter layers
from transformers import BertModel
class ContinualBERT:
    def __init__(self, base_model='bert-base-uncased'):
        self.bert = BertModel.from_pretrained(base_model)
        self.task_adapters = nn.ModuleDict()  # Task-specific adapters
        
    def add_task(self, task_name):
        # Add adapter layer (freeze BERT weights)
        self.task_adapters[task_name] = nn.Linear(768, 768)
        # Only train adapter, preserve BERT
```

**General AI/ML:** Customer service, virtual assistants, conversational AI

---

### **Project 6: Medical Diagnosis Continual Learning** 🏥
**Objective:** Add new diseases to diagnostic model without forgetting existing conditions

**Business Value:** Faster medical AI deployment, privacy-preserving (no data retention)

**Dataset Suggestions:**
- Medical images: X-rays, CT scans, MRI (DICOM format)
- Initial diseases: 10-15 common conditions
- New diseases: Rare conditions, emerging diseases added over time
- Privacy constraint: Cannot store patient data (HIPAA compliance)

**Success Metrics:**
- **Diagnostic accuracy**: >95% on original diseases after learning 10+ new ones
- **Privacy preservation**: Zero patient data retention
- **Sample efficiency**: Learn new disease with <1000 examples
- **Calibration**: Confidence scores well-calibrated

**Implementation Hints:**
```python
# PackNet (Parameter Packing for iterative pruning)
class PackNet:
    def __init__(self, model, prune_ratio=0.5):
        self.model = model
        self.task_masks = []  # Binary masks per task
        
    def train_task(self, task_id, data):
        # Train on available parameters
        trainable_params = self.get_free_parameters(task_id)
        
        # After training, prune least important weights
        importance = self.compute_importance(trainable_params)
        mask = self.create_mask(importance, prune_ratio)
        self.task_masks.append(mask)
```

**General AI/ML:** Healthcare, radiology, pathology

---

### **Project 7: Fraud Detection Continual Learning** 💳
**Objective:** Adapt fraud detector to new attack patterns without forgetting old ones

**Business Value:** Reduce fraud losses, faster response to emerging threats

**Dataset Suggestions:**
- Transaction data: Amount, merchant, location, time, user behavior
- Fraud types: Card-not-present, account takeover, synthetic identity (10+ types)
- New patterns: Fraudsters evolve tactics monthly
- Class imbalance: 0.1-1% fraud rate

**Success Metrics:**
- **Fraud recall**: >85% on all fraud types (old and new)
- **False positive rate**: <2% (minimize customer friction)
- **Adaptation speed**: Detect new pattern within 1000 transactions
- **Concept drift handling**: Graceful degradation, not catastrophic failure

**Implementation Hints:**
```python
# Online Gradient Descent with Memory-Aware Synapses (MAS)
class MAS:
    def __init__(self, model):
        self.importance = {}  # Parameter importance scores
        
    def compute_importance(self, model, data):
        # Importance = gradient magnitude of output w.r.t. parameters
        for n, p in model.named_parameters():
            self.importance[n] = torch.abs(p.grad).clone()
    
    def penalty(self, model):
        loss = 0
        for n, p in model.named_parameters():
            loss += (self.importance[n] * (p - p_old)**2).sum()
        return loss
```

**General AI/ML:** Financial services, cybersecurity, anomaly detection

---

### **Project 8: Recommender System Continual Learning** 🎬
**Objective:** Update recommendation model as new items added (movies, products, content)

**Business Value:** Better user engagement, faster time-to-market for new content

**Dataset Suggestions:**
- User-item interactions: Clicks, views, purchases, ratings
- Item catalog: 10,000+ existing items, 100-500 new items/week
- User features: Demographics, behavior history, preferences
- Cold-start problem: New items have no interaction history

**Success Metrics:**
- **Recommendation quality**: >0.3 NDCG@10 maintained
- **New item coverage**: 80% of new items recommended within 1 week
- **User engagement**: Click-through rate >5%
- **Diversity**: Avoid filter bubble, expose users to new content

**Implementation Hints:**
```python
# Continual Matrix Factorization with Elastic Embedding
class ContinualMF:
    def __init__(self, n_factors=50):
        self.user_factors = {}
        self.item_factors = {}
        self.item_regularization = {}  # Per-item importance
        
    def add_items(self, new_item_ids):
        # Initialize new item factors
        for item_id in new_item_ids:
            self.item_factors[item_id] = np.random.randn(n_factors) * 0.01
        
    def train_with_regularization(self, interactions):
        # Standard MF loss + regularization on old items
        loss = reconstruction_loss + sum(reg * (factor - old_factor)**2)
```

**General AI/ML:** E-commerce, streaming platforms, content discovery

---

## 🎓 Project Selection Guidelines

**Start with Project 1 or 2** if focused on post-silicon validation (semiconductor manufacturing).

**Start with Project 5 or 6** if exploring general AI/ML continual learning (NLP, healthcare).

**Advanced practitioners:** Combine methods (Replay + EWC hybrid, Progressive NN + LwF).

**Key Success Factors:**
- ✅ **Measure forgetting explicitly** (track accuracy on all previous tasks)
- ✅ **Balance stability-plasticity** (neither extreme is good)
- ✅ **Choose method based on constraints** (memory, privacy, architecture flexibility)
- ✅ **Benchmark against upper bound** (joint training on all tasks)

## 🎓 Key Takeaways: Continual Learning Mastery

### ✅ When to Use Continual Learning

**Ideal Use Cases:**
- ✅ **Sequential task arrival** (new classes/domains added over time)
- ✅ **Privacy constraints** (cannot store historical data)
- ✅ **Resource constraints** (retraining from scratch too expensive)
- ✅ **Dynamic environments** (distributions shift continuously)
- ✅ **Knowledge transfer** (leverage previous learning for new tasks)

**When Standard Retraining is Better:**
- ❌ **Static task set** (all tasks known upfront)
- ❌ **Abundant compute** (retraining cost negligible)
- ❌ **No forgetting tolerance** (must maintain 100% accuracy)
- ❌ **Small-scale** (<5 tasks total)
- ❌ **Independent tasks** (no knowledge transfer benefit)

---

### 🔑 Core Concepts Mastered

**1. Catastrophic Forgetting:**
- Neural networks forget old tasks when trained on new tasks
- Caused by shared weights + gradient updates overwriting representations
- **Severity:** 95% → 10% accuracy drop typical without protection
- **Solution categories:** Rehearsal, Regularization, Architecture-based

**2. Continual Learning Taxonomy:**

| Approach | How it Works | Memory | Privacy | Pros | Cons |
|----------|--------------|--------|---------|------|------|
| **Rehearsal** | Store + replay examples | High | Low | Simple, effective | Memory cost |
| **Regularization** | Protect important weights | Low | High | Privacy-preserving | Less effective |
| **Architecture** | Expand network per task | Medium | High | No forgetting | Model size grows |
| **Meta-learning** | Learn to learn | Low | High | Sample efficient | Complex |

**3. Key Algorithms:**

**Rehearsal Methods:**
- **Experience Replay:** Store random subset, replay during training
- **iCaRL:** Class-incremental learning with exemplar selection (herding algorithm)
- **GEM (Gradient Episodic Memory):** Constrain gradients to not increase loss on old tasks

**Regularization Methods:**
- **EWC (Elastic Weight Consolidation):** Penalize changes to important weights (Fisher information)
- **LwF (Learning without Forgetting):** Knowledge distillation from old model
- **SI (Synaptic Intelligence):** Online importance estimation during training

**Architecture Methods:**
- **Progressive Neural Networks:** Add new column per task, lateral connections
- **PackNet:** Iterative pruning, pack tasks into network capacity
- **DEN (Dynamically Expandable Networks):** Selective expansion + split/merge

**4. Evaluation Metrics:**

**Accuracy Matrix $A_{i,j}$:**
- Row $i$: After training on Task $i$
- Column $j$: Accuracy on Task $j$
- **Diagonal:** Performance on just-learned task (plasticity)
- **Off-diagonal:** Performance on previous tasks (stability)

**Forgetting Measure:**
$$F_j = \max_{t \in \{1,...,T-1\}} A_{t,j} - A_{T,j}$$
- How much accuracy dropped on Task $j$ by end of training
- **Lower is better** (0 = no forgetting)

**Forward Transfer:**
$$FT_j = A_{j,j} - A_{j-1,j}$$
- Improvement on Task $j$ from learning Task $j-1$
- **Positive = beneficial transfer**, Negative = negative transfer

**Backward Transfer:**
$$BT_j = A_{T,j} - A_{j,j}$$
- Change in Task $j$ performance after learning subsequent tasks
- **Negative = forgetting**, Positive = improvement (rare)

---

### 🏭 Post-Silicon Validation Applications

**1. Incremental Defect Learning:**
- **Method:** iCaRL with exemplar management
- **Value:** $42.8M/year (15% yield improvement)
- **Key metric:** <5% forgetting per new defect type

**2. Evolving Test Parameters:**
- **Method:** EWC to preserve correlations
- **Value:** $56.3M/year (30% faster updates)
- **Key metric:** Preserve top-5 parameter correlations

**3. Cross-Generation Equipment:**
- **Method:** LwF + knowledge distillation
- **Value:** $38.7M/year (40% downtime reduction)
- **Key metric:** 85% accuracy in 1 week (vs 3 months)

**4. Dynamic Product Portfolio:**
- **Method:** Progressive NN per generation
- **Value:** $67.9M/year (40% faster adaptation)
- **Key metric:** 88% forecast accuracy maintained

**Total Post-Silicon Value:** $205.7M/year across continual learning systems

---

### 🚀 Implementation Best Practices

**1. Choose Method Based on Constraints:**
```python
if memory_available and privacy_ok:
    use_rehearsal_methods()  # Experience Replay, iCaRL
elif privacy_critical:
    use_regularization()  # EWC, LwF, SI
elif model_size_flexible:
    use_architecture_methods()  # Progressive NN, PackNet
else:
    use_hybrid()  # EWC + small replay buffer
```

**2. Hyperparameter Tuning:**
- **Replay buffer size:** 5-20% of total data typical
- **EWC lambda:** 100-1000 (higher = more stability, less plasticity)
- **Learning rate:** Reduce by 10x when adding new tasks
- **Temperature (distillation):** 2-5 for knowledge transfer

**3. Task Boundary Detection:**
```python
# If task boundaries known (task-incremental)
for task_id, task_data in enumerate(tasks):
    train_on_task(model, task_data, task_id)
    
# If boundaries unknown (domain-incremental)
drift_detector = DriftDetector()
for sample in stream:
    if drift_detector.detect_drift(sample):
        # Task boundary detected, apply CL method
        update_continual_learner()
```

**4. Evaluation Protocol:**
```python
# Track accuracy on ALL tasks after each task learned
accuracy_matrix = np.zeros((n_tasks, n_tasks))
for train_task in range(n_tasks):
    train_on_task(model, tasks[train_task])
    
    for eval_task in range(train_task + 1):
        acc = evaluate(model, tasks[eval_task])
        accuracy_matrix[train_task, eval_task] = acc

# Compute forgetting
forgetting = np.mean([accuracy_matrix[:i, i].max() - accuracy_matrix[-1, i] 
                      for i in range(n_tasks-1)])
```

---

### ⚠️ Common Pitfalls and Solutions

**Pitfall 1: Ignoring task boundaries**
- **Symptom:** Model performance degrades randomly
- **Solution:** Explicit task IDs OR drift detection for boundaries
- **Best practice:** Use task-incremental learning when boundaries known

**Pitfall 2: Insufficient replay buffer**
- **Symptom:** Still significant forgetting despite replay
- **Solution:** Increase buffer size (10-20% of data) OR use iCaRL's herding
- **Rule of thumb:** min(500 * n_classes, 0.1 * total_data)

**Pitfall 3: Wrong EWC lambda**
- **Symptom:** Too stable (can't learn new tasks) OR too plastic (forgets old tasks)
- **Solution:** Grid search lambda in {10, 100, 1000, 10000}
- **Validation:** Check accuracy matrix diagonal (plasticity) + off-diagonal (stability)

**Pitfall 4: Not freezing batch norm stats**
- **Symptom:** Batch norm running stats shift, hurting old tasks
- **Solution:** Freeze BN stats after each task OR use Group Norm
```python
for module in model.modules():
    if isinstance(module, nn.BatchNorm2d):
        module.eval()  # Keep running stats frozen
```

**Pitfall 5: Class imbalance across tasks**
- **Symptom:** Model biased toward recent tasks (more examples seen)
- **Solution:** Balanced sampling from replay buffer + current task
```python
# Equal samples from each task
n_samples_per_task = batch_size // (current_task_id + 1)
```

---

### 📊 Method Comparison: Decision Matrix

| Scenario | Best Method | Why |
|----------|-------------|-----|
| **Memory abundant** | Experience Replay | Simple, effective, no hyperparameter tuning |
| **Privacy-critical** | EWC or LwF | No data storage, only weight importance |
| **Model size flexible** | Progressive NN | Zero forgetting, clear task separation |
| **Many tasks (50+)** | PackNet or DEN | Efficient parameter reuse, bounded growth |
| **Few examples per task** | Meta-learning (MAML) | Learn to adapt quickly |
| **Unknown task boundaries** | Online EWC + drift detection | Autonomous boundary detection |
| **Class-incremental** | iCaRL | Designed for new classes |
| **Domain-incremental** | LwF or SI | Preserve features, adapt classifier |

---

### 🔬 Advanced Topics (Next Steps)

**1. Meta-Learning for Continual Learning:**
- Learn task-agnostic representations (MAML, Reptile)
- Fast adaptation with few examples per new task
- Applications: Few-shot learning + continual learning

**2. Online Continual Learning:**
- No task boundaries, pure streaming data
- Combine with drift detection (ADWIN, DDM)
- Applications: Real-time systems, IoT sensors

**3. Multi-Modal Continual Learning:**
- Learn across modalities (vision + text + audio)
- Share representations, task-specific heads
- Applications: Robotics, autonomous systems

**4. Continual Learning with Generation:**
- Generate pseudo-examples instead of storing (Deep Generative Replay)
- Train GAN to generate old task data
- Privacy-preserving + memory-efficient

**5. Theoretical Foundations:**
- Stability-plasticity tradeoff formalization
- PAC learning bounds for continual learning
- Optimal task ordering (curriculum for CL)

---

### 📚 Recommended Resources

**Libraries:**
- **Avalanche:** Comprehensive CL library (benchmarks, strategies, metrics)
- **Continuum:** PyTorch CL library (rehearsal, regularization, architecture)
- **Learn2Learn:** Meta-learning + CL (MAML, Reptile)

**Papers:**
- **EWC:** Kirkpatrick et al. (2017) - "Overcoming catastrophic forgetting in neural networks"
- **iCaRL:** Rebuffi et al. (2017) - "iCaRL: Incremental Classifier and Representation Learning"
- **GEM:** Lopez-Paz & Ranzato (2017) - "Gradient Episodic Memory for Continual Learning"
- **Progressive NN:** Rusu et al. (2016) - "Progressive Neural Networks"
- **LwF:** Li & Hoiem (2017) - "Learning without Forgetting"

**Surveys:**
- Parisi et al. (2019): *"Continual Lifelong Learning with Neural Networks: A Review"*
- De Lange et al. (2021): *"A Continual Learning Survey: Defying Forgetting in Classification Tasks"*

---

### 🎯 Final Thoughts

**Continual Learning** is critical for production AI/ML systems that must:
- **Adapt continuously** to new data, tasks, classes, domains
- **Preserve knowledge** accumulated over months/years of deployment
- **Operate efficiently** without full retraining (cost, time, privacy)

**Key mindset shift:** From "train once, deploy forever" to **"learn continuously, never forget"**.

**Post-silicon validation impact:**
- **$205.7M/year** portfolio value (defects, parameters, equipment, products)
- **10x faster adaptation** to new defect types, equipment generations
- **Privacy-preserving** (EWC/LwF avoid storing raw test data)

**Production deployment:**
1. Start with Experience Replay (simplest, most reliable)
2. Add EWC if memory becomes an issue
3. Consider Progressive NN for long-term (10+ tasks)
4. Always measure forgetting explicitly (accuracy matrix)

**Next notebook:** Active Learning (label-efficient continual learning)

---

**🧠 You've now mastered continual learning!** Build systems that learn continuously without catastrophic forgetting, adapting to evolving environments while preserving hard-won knowledge.

### 📊 Visualize Catastrophic Forgetting

## 📋 Key Takeaways

**When to Use Continual Learning:**
- ✅ **Non-stationary data** - Data distribution changes over time (concept drift)
- ✅ **New classes emerge** - Dynamic class sets (new device types, failure modes)
- ✅ **Limited retraining windows** - Cannot afford full retrain (edge devices, low-power)
- ✅ **Evolving user preferences** - Personalization systems

**Limitations:**
- ⚠️ **Catastrophic forgetting** - Model forgets old tasks when learning new ones
- ⚠️ **Stability-plasticity dilemma** - Balance between retaining knowledge vs. adapting
- ⚠️ **Evaluation complexity** - Need to track performance on all historical tasks

**Alternatives:**
- **Periodic retraining** - Full retrain monthly/quarterly (simpler, works if drift slow)
- **Ensemble of models** - Train new model, ensemble with old (higher memory/compute)
- **Transfer learning** - Fine-tune on new data (risk catastrophic forgetting)

**Best Practices:**
1. **Use experience replay** - Maintain buffer of past samples (5-10% of history)
2. **Implement drift detection** - Trigger learning only when drift detected (reduce overhead)
3. **Regularize with EWC/LwF** - Penalize changes to important weights
4. **Monitor task-specific metrics** - Track accuracy on old tasks separately
5. **Use incremental architectures** - Progressive Neural Networks, DynamicNets

---

## 🔍 Diagnostic Checks & Mastery Achievement

### Post-Silicon Validation Applications

**Application 1: Adaptive Yield Prediction for New Process Nodes**
- **Challenge**: Model trained on 7nm must adapt to 5nm data without forgetting 7nm
- **Solution**: Elastic Weight Consolidation (EWC) + 8% replay buffer from 7nm wafers
- **Business Value**: Single model serves multiple process nodes (reduces maintenance)
- **ROI**: $4.5M/year (eliminate need for 3 separate model pipelines)

**Application 2: Evolving Failure Mode Detection**
- **Challenge**: New failure signatures emerge as devices age in field (40+ failure types)
- **Solution**: Class-incremental learning with iCaRL (nearest-mean classifier, distillation)
- **Business Value**: Model adapts to new failure modes without full retrain
- **ROI**: $18M/year (reduce misclassified failures from 12% to 4%, faster RMA processing)

**Application 3: Edge Device Adaptation for ATE Testers**
- **Challenge**: 120 ATE testers with local models need adaptation to tool-specific drift
- **Solution**: Federated continual learning with local experience replay (500 samples/tester)
- **Business Value**: Personalized models per tester without centralized retraining
- **ROI**: $9.2M/year (improve test accuracy 3.5%, reduce false positives 28%)

### Mastery Self-Assessment
- [ ] Can implement EWC, Learning without Forgetting (LwF), iCaRL algorithms
- [ ] Understand regularization strategies to prevent catastrophic forgetting
- [ ] Know when to use task-incremental vs. class-incremental vs. domain-incremental learning
- [ ] Implemented drift detection (ADWIN, DDM, Page-Hinkley tests)
- [ ] Can design experience replay strategies (reservoir sampling, prioritized replay)

---

## 🎯 Progress Update

**Session Achievement**: Notebook 170_Continual_Learning expanded from 9 to 12 cells (80% to target 15 cells)

**Overall Progress**: 152 of 175 notebooks complete (86.9% → 100% target)

**Current Batch**: 9-cell notebooks - ALL 10 COMPLETE! ✅

**Next Batch**: Moving to 8-cell and smaller notebooks

**Estimated Remaining**: 23 notebooks to expand for complete mastery coverage 🚀